![ga4](https://www.google-analytics.com/collect?v=2&tid=G-6VDTYWLKX6&cid=1&en=page_view&sid=1&dl=statmike%2Fvertex-ai-mlops%2FApplied+GenAI%2FVertex+AI+Search&dt=Vertex+AI+Search+Document+Q%26A+Using+Extractive+Segments+-+MLB+Rules+For+Baseball.ipynb)

# UmpireBot - MLB Rules For Baseball

Based on [Vertex AI Search](https://cloud.google.com/enterprise-search) (Formerly: Generative AI App Builder Enterprise Search).

**What?**

Ask questions of the rules for MLB and get answers with specific references to official rules.

**Sources:**

The official MLB rules are at the links below:
- [MLB 2022 Rule Book](https://img.mlbstatic.com/mlb-images/image/upload/mlb/hhvryxqioipb87os1puw.pdf)
- [MLB 2023 Rule Book](https://img.mlbstatic.com/mlb-images/image/upload/mlb/wqn5ah4c3qtivwx3jatm.pdf)

**Tools:**
- Vertex AI Search
  - This [link](https://cloud.google.com/generative-ai-app-builder/docs/create-data-store-es#cloud-storage) will provide the steps to create a datastore using unstructured data(pdf is used in this example) which will be stored in Cloud Storage.
  - Vertex AI Search Setup for this workflow can be found [here](./vertex_search_setup.md)
  - Vertex AI Search Client
      - [Python Client for Discovery Engine API](https://cloud.google.com/python/docs/reference/discoveryengine/latest)
- Vertex AI GenAI Language Model API
  - Vertex AI Client
      - [Vertex AI SDK for Python](https://cloud.google.com/python/docs/reference/aiplatform/latest)
  - `vertexai.language_models.TextGenerationModel.from_pretrained('text-bison@latest')`

---

**Google Cloud Vertex AI Generative AI Support**

Vertex AI Generative AI gives access to Google's large genearative AI models and also enables you to test, tune, and deploy them for your applications. Get an overview [here](https://cloud.google.com/vertex-ai/docs/generative-ai/learn/overview).

---

**How:**

- Vertex AI Search creates three components: Apps, Engines and Datastores
  - More information about App and Datastore can be found [here](https://cloud.google.com/generative-ai-app-builder/docs/create-datastore-ingest)
      - Datastores from web sites, BigQuery Tables, GCS Buckets, and much more!
- Ask a question
  - Query the datastore using the Python Client API
  - API will search the datastore for related information in the documents using vector similarity
  - Retrieve the content associated from Datastore
  - Prepare a prompt to answer the question using the retrieved search results as context
- Present the response as an answer with links to the related documents (sections of the document(s)).

**What is Unique?**

An LLM is likely unexposed to a users private content.  This appoach constructs a summarization prompt for an LLM by first retriving context for the question from the users documents using Vertex AI Search datastores as the source.  This also allows the response to be accompanied by direct reference to the users documentation used in the prompt.

**Notes**

- An LLM is likely trained on many sources that probably include lots of general knowledge, even information like what is used here.  It is also likely knowledgable of past outdated information which can be a benefit - or detriment - to accuracy.  This approach directly uses the version of the document that currently applies.

**Prerequisites**

This notebook uses Vertex AI and Vertex AI Search (Generative AI App Builder) for processing data while also using Google Cloud Storage for data storage and retrieval.  If you are running this notebook from Colab or another environment where your user id is authenticated then your account will need roles/permissions that allow working with these services.  If you are running this from a Vertex AI Workbench Notebook instance then it is running as a service account which will need the roles/permission that allow working with these services.

This notebook requires an App and Data Store created with the PDF downloaded from MLB website mentioned above.
- Vertex AI Search Setup for this workflow can be found [here](./vertex_search_setup.md)


**References**

- [Vertex AI GenAI Studio](https://cloud.google.com/vertex-ai/docs/generative-ai/learn/overview)
    - Vertex AI [Python Client](https://cloud.google.com/python/docs/reference/aiplatform/latest)
- Vertex AI Search [Python Client](https://cloud.google.com/generative-ai-app-builder/docs/preview-search-results?_ga=2.154471792.-952990647.1684293017)
   

---
## Colab Setup

To run this notebook in Colab click [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/statmike/vertex-ai-mlops/blob/main/Applied%20GenAI/Vertex%20AI%20Search/Vertex%20AI%20Search%20Document%20Q%26A%20Using%20Extractive%20Segments%20-%20MLB%20Rules%20For%20Baseball.ipynb) and run the cells in this section.  Otherwise, skip this section.

This cell will authenticate to GCP (follow prompts in the popup).

In [1]:
PROJECT_ID = 'statmike-mlops-349915' # replace with project ID

In [2]:
try:
    import google.colab
    from google.colab import auth
    auth.authenticate_user()
    !gcloud config set project {PROJECT_ID}
except Exception:
    pass

---
## Installs and API Enablement

The clients packages may need installing in this environment. 

### Installs (If Needed)

In [3]:
# tuples of (import name, install name)
packages = [
    ('google.cloud.aiplatform', 'google-cloud-aiplatform'),
    ('google.cloud.discoveryengine', 'google-cloud-discoveryengine')
]

import importlib
install = False
for package in packages:
    if not importlib.util.find_spec(package[0]):
        print(f'installing package {package[1]}')
        install = True
        !pip install {package[1]} -U -q --user

### API Enablement

In [4]:
!gcloud services enable aiplatform.googleapis.com

### Restart Kernel (If Installs Occured)

After a kernel restart the code submission can start with the next cell after this one.

In [5]:
if install:
    import IPython
    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

---
## Setup

Inputs

In [6]:
project = !gcloud config get-value project
PROJECT_ID = project[0]
PROJECT_ID

'statmike-mlops-349915'

In [7]:
REGION = 'us-central1'
SERIES = 'applied-genai-vertex-ai-search'
EXPERIMENT = 'mlb-rules'

# VERTEX SEARCH PARAMETERS
VS_LOCATION = 'global'
VS_DATASTORE_ID = 'sports_1699306284060'

Packages

In [8]:
import IPython

import vertexai.language_models
from google.cloud import aiplatform
import google.cloud.discoveryengine_v1 as discoveryengine
import google.cloud.discoveryengine_v1alpha as discoveryengine_alpha

2023-11-07 16:57:31.867108: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Clients

In [9]:
# vertex ai clients
vertexai.init(project = PROJECT_ID, location = REGION)
aiplatform.init(project = PROJECT_ID, location = REGION)

# vertex search clients
vertex_search = discoveryengine.SearchServiceClient(
    client_options = dict(api_endpoint = (f'{VS_LOCATION}-' if VS_LOCATION != 'global' else '') + 'discoveryengine.googleapis.com')
)
vertex_search_alpha = discoveryengine_alpha.SearchServiceClient(
    client_options = dict(api_endpoint = (f'{VS_LOCATION}-' if VS_LOCATION != 'global' else '') + 'discoveryengine.googleapis.com')
)

## Using Vertex AI Search 

The client for Vertex AI Search can be used to extract a number of helpful items for use with an LLM as context.

Documentation Links:
- [Search Results](https://cloud.google.com/generative-ai-app-builder/docs/preview-search-results)
- [Snippets](https://cloud.google.com/generative-ai-app-builder/docs/snippets#snippets)
- [Extractive Answers](https://cloud.google.com/generative-ai-app-builder/docs/snippets#extractive-answers)
- [Extractive Segments](https://cloud.google.com/generative-ai-app-builder/docs/snippets#extractive-segments)
- [Search Summaries](https://cloud.google.com/generative-ai-app-builder/docs/get-search-summaries)

API Reference (Python):
- [discoveryengine.SearchServiceClient()](https://cloud.google.com/python/docs/reference/discoveryengine/latest/google.cloud.discoveryengine_v1.services.search_service.SearchServiceClient#google_cloud_discoveryengine_v1_services_search_service_SearchServiceClient_search)
    - [.search()](https://cloud.google.com/python/docs/reference/discoveryengine/latest/google.cloud.discoveryengine_v1.services.search_service.SearchServiceClient#google_cloud_discoveryengine_v1_services_search_service_SearchServiceClient_search)
        - request = [discoveryengine.types.SearchRequest()](https://cloud.google.com/python/docs/reference/discoveryengine/latest/google.cloud.discoveryengine_v1.types.SearchRequest)
            - `page_size` = number of documents to return
            - `serving_config` = [discoveryengine.search_service.SearchServiceClient.serving_config_path](https://cloud.google.com/python/docs/reference/discoveryengine/latest/google.cloud.discoveryengine_v1.services.search_service.SearchServiceClient#google_cloud_discoveryengine_v1_services_search_service_SearchServiceClient_serving_config_path)
            - content_search_spec = [discoveryengine.types.SearchRequest.ContentSearchSpec()](https://cloud.google.com/python/docs/reference/discoveryengine/latest/google.cloud.discoveryengine_v1.types.SearchRequest.ContentSearchSpec)
                - `snippet_spec` = [discoveryengine.SearchRequest.ContentSearchSpec.SnippetSpec()](https://cloud.google.com/python/docs/reference/discoveryengine/latest/google.cloud.discoveryengine_v1.types.SearchRequest.ContentSearchSpec.SnippetSpec)
                    - if not specified, then snippets are not included in search response
                    - `return_snippet` = True
                - `summary_spec` = [discoveryengine.SearchRequest.ContentSearchSpec.SummarySpec()](https://cloud.google.com/python/docs/reference/discoveryengine/latest/google.cloud.discoveryengine_v1.types.SearchRequest.ContentSearchSpec.SummarySpec)
                    - if not specified, then summaries are not included in search response
                    - `summery_result_count` = the number of results to generate the summary from - max is 5
                    - `include_citations` = True/False, to include in-line citation numbers
                    - `ignore_adversarial_query` = True/False, to ignore/skip adversarial queries
                    - `ignore_non_summary_seeking_query` = True/False, to ignore fact seeking queries like "help desk phone number"
            - query_expansition_spec = [discoveryengine.types.SearchRequest.QueryExpansionSpec.Condition.](https://cloud.google.com/python/docs/reference/discoveryengine/latest/google.cloud.discoveryengine_v1.types.SearchRequest.QueryExpansionSpec)
                - configure expansion of query:
                    - `.CONDITION_UNSPECIFIED` defaults to `.DISABLED`
                    - `.DISABLED` only the exact search is used even if no response if generated
                    - `.AUTO` automatic expansion of the query by the Search API
            - spell_correction_spec = [discoveryengine.types.SearchRequest.SpellCorrectionSpec.Mode.](https://cloud.google.com/python/docs/reference/discoveryengine/latest/google.cloud.discoveryengine_v1.types.SearchRequest.SpellCorrectionSpec)
                - spelling correction mode:
                    - `.MODE_UNSPECIFIED` defaults to `.AUTO`
                    - `.SUGGESTION_ONLY` - returns a corrected query suggestion but does not use it for search
                    - `.AUTO` - uses correct query suggestion for search
            - much more!
- Responses From `.search()` are of the form [discoveryengine.search_service.pagers.SearchPager()](https://cloud.google.com/python/docs/reference/discoveryengine/latest/google.cloud.discoveryengine_v1.services.search_service.pagers.SearchPager)
    - Which contains `.results` object that is an iterable set of [discoveryengine.search_service.SearchResponse.SearchResult()](https://cloud.google.com/python/docs/reference/discoveryengine/latest/google.cloud.discoveryengine_v1.types.SearchResponse.SearchResult)
        - has attribute `document` of type [discoveryengine.types.Document](https://cloud.google.com/python/docs/reference/discoveryengine/latest/google.cloud.discoveryengine_v1.types.Document)
            - for a data store of document in GCS these attributes are helpful:
                - `derived_struct_data`

In [116]:
question = "What is the size of a base?"

Setup the `serving_config`:
- [discoveryengine.search_service.SearchServiceClient.serving_config_path](https://cloud.google.com/python/docs/reference/discoveryengine/latest/google.cloud.discoveryengine_v1.services.search_service.SearchServiceClient#google_cloud_discoveryengine_v1_services_search_service_SearchServiceClient_serving_config_path)

In [117]:
serving_config = vertex_search.serving_config_path(
    project = PROJECT_ID,
    location = VS_LOCATION,
    data_store = VS_DATASTORE_ID,
    serving_config = 'default_config'
)

### Search Results

Documentation [Search Results](https://cloud.google.com/generative-ai-app-builder/docs/preview-search-results)

In [118]:
response = vertex_search.search(
    request = discoveryengine.SearchRequest(
        query = question,
        page_size = 10,
        serving_config = serving_config,
        content_search_spec = discoveryengine.SearchRequest.ContentSearchSpec(
            snippet_spec = discoveryengine.SearchRequest.ContentSearchSpec.SnippetSpec(
                return_snippet = False
            ),
            summary_spec = discoveryengine.SearchRequest.ContentSearchSpec.SummarySpec(
                summary_result_count = 5,
                include_citations = True,
                ignore_adversarial_query = True,
                ignore_non_summary_seeking_query = False
            )
        ),
        query_expansion_spec = discoveryengine.SearchRequest.QueryExpansionSpec(
            condition = discoveryengine.SearchRequest.QueryExpansionSpec.Condition.AUTO
        ),
        spell_correction_spec = discoveryengine.SearchRequest.SpellCorrectionSpec(
            mode = discoveryengine.SearchRequest.SpellCorrectionSpec.Mode.AUTO
        )
    )
)

In [119]:
text = ''
for r, result in enumerate(response.results):
    if r > 0: text += '\n\n\n'
    doc = dict(result.document.derived_struct_data)
    text += f"Source Document {r+1}:\n\tName: {doc['title']}\n\tLink: {doc['link']}"
print(text)
search_results = text

Source Document 1:
	Name: mlb_2022_rules
	Link: gs://statmike-mlops-349915/applied-genai/vertex-ai-search/mlb_2022_rules.pdf


Source Document 2:
	Name: mlb_2023_rules
	Link: gs://statmike-mlops-349915/applied-genai/vertex-ai-search/mlb_2023_rules.pdf


### Snippets

[Snippets](https://cloud.google.com/generative-ai-app-builder/docs/snippets#snippets)

In [120]:
response = vertex_search.search(
    request = discoveryengine.SearchRequest(
        query = question,
        page_size = 10,
        serving_config = serving_config,
        content_search_spec = discoveryengine.SearchRequest.ContentSearchSpec(
            snippet_spec = discoveryengine.SearchRequest.ContentSearchSpec.SnippetSpec(
                return_snippet = True
            ),
            summary_spec = discoveryengine.SearchRequest.ContentSearchSpec.SummarySpec(
                summary_result_count = 5,
                include_citations = True,
                ignore_adversarial_query = True,
                ignore_non_summary_seeking_query = False
            )
        ),
        query_expansion_spec = discoveryengine.SearchRequest.QueryExpansionSpec(
            condition = discoveryengine.SearchRequest.QueryExpansionSpec.Condition.AUTO
        ),
        spell_correction_spec = discoveryengine.SearchRequest.SpellCorrectionSpec(
            mode = discoveryengine.SearchRequest.SpellCorrectionSpec.Mode.AUTO
        )
    )
)

In [121]:
text = ''
for r, result in enumerate(response.results):
    if r > 0: text += '\n\n\n'
    doc = dict(result.document.derived_struct_data)
    text += f"Source Document {r+1}:\n\tName: {doc['title']}\n\tLink: {doc['link']}"
    if 'snippets' in doc.keys():
        for s, snippet in enumerate(doc['snippets']):
            if snippet['snippet_status'] == 'SUCCESS':
                text += f"\n\tSnippet {s+1}: {snippet['snippet']}"
print(text)
snippets_results = text

Source Document 1:
	Name: mlb_2022_rules
	Link: gs://statmike-mlops-349915/applied-genai/vertex-ai-search/mlb_2022_rules.pdf
	Snippet 1: ... <b>size</b> and shape of the grassed and bare areas of its playing field. NOTE: (a) ... The second <b>base</b> bag shall be centered on second <b>base</b>. The bags shall be 15&nbsp;...


Source Document 2:
	Name: mlb_2023_rules
	Link: gs://statmike-mlops-349915/applied-genai/vertex-ai-search/mlb_2023_rules.pdf
	Snippet 1: It is recommended that the distance from home <b>base</b> to the backstop, and from the <b>base</b> lines to the nearest fence, stand or other obstruction on foul territory&nbsp;...


In [122]:
response.summary

summary_text: "In Major League Baseball, home base is a five-sided slab of whitened rubber that is 17 inches square [1, 2]. Two of the corners are removed so that one edge is 17 inches long, two adjacent sides are 8\302\275 inches, and the remaining two sides are 12 inches [1, 2]."

### Extractive Answers

Sections of text derived verbatim from documents in the search result.  Answers are shorter than segments (next section).

[Extractive Answers](https://cloud.google.com/generative-ai-app-builder/docs/snippets#extractive-answers)

For now, Nov 2023, this feature using the [discoveryengine_v1alpha](https://cloud.google.com/python/docs/reference/discoveryengine/latest/google.cloud.discoveryengine_v1alpha.services.search_service.SearchServiceClient) client in order to get the `extractive_content_spec` attribute for the [discoveryengine_v1alpha1.types.SearchRequest.ContentSearchSpec()](https://cloud.google.com/python/docs/reference/discoveryengine/latest/google.cloud.discoveryengine_v1alpha.types.SearchRequest.ContentSearchSpec).

In [123]:
response = vertex_search_alpha.search(
    request = discoveryengine_alpha.SearchRequest(
        query = question,
        page_size = 10,
        serving_config = serving_config,
        content_search_spec = discoveryengine_alpha.SearchRequest.ContentSearchSpec(
            snippet_spec = discoveryengine_alpha.SearchRequest.ContentSearchSpec.SnippetSpec(
                return_snippet = True
            ),
            summary_spec = discoveryengine_alpha.SearchRequest.ContentSearchSpec.SummarySpec(
                summary_result_count = 5,
                include_citations = True,
                ignore_adversarial_query = True,
                ignore_non_summary_seeking_query = False
            ),
            extractive_content_spec = discoveryengine_alpha.SearchRequest.ContentSearchSpec.ExtractiveContentSpec(
                max_extractive_answer_count = 5
            )
        ),
        query_expansion_spec = discoveryengine_alpha.SearchRequest.QueryExpansionSpec(
            condition = discoveryengine_alpha.SearchRequest.QueryExpansionSpec.Condition.AUTO
        ),
        spell_correction_spec = discoveryengine_alpha.SearchRequest.SpellCorrectionSpec(
            mode = discoveryengine_alpha.SearchRequest.SpellCorrectionSpec.Mode.AUTO
        )
    )
)

In [124]:
text = ''
for r, result in enumerate(response.results):
    if r > 0: text += '\n\n\n'
    doc = dict(result.document.derived_struct_data)
    text += f"Source Document {r+1}:\n\tName: {doc['title']}\n\tLink: {doc['link']}"
    if 'snippets' in doc.keys():
        for s, snippet in enumerate(doc['snippets']):
            if snippet['snippet_status'] == 'SUCCESS':
                text += f"\n\tSnippet {s+1}: {snippet['snippet']}"
    if 'extractive_answers' in doc.keys():
        for a, answer in enumerate(doc['extractive_answers']):
            text += f"\n\tAnswer {a+1} (page = {answer['pageNumber']}): {answer['content']}"
print(text)
answers_results = text

Source Document 1:
	Name: mlb_2022_rules
	Link: gs://statmike-mlops-349915/applied-genai/vertex-ai-search/mlb_2022_rules.pdf
	Snippet 1: ... <b>size</b> and shape of the grassed and bare areas of its playing field. NOTE: (a) ... The second <b>base</b> bag shall be centered on second <b>base</b>. The bags shall be 15&nbsp;...
	Answer 1 (page = 15): 2.02 Home Base Home base shall be marked by a five-sided slab of whitened rubber. It shall be a 17-inch square with two of the corners removed so that one edge is 17 inches long, two adjacent sides are 8½ inches and the remaining two sides are 12 inches and set at an angle to make a point.
	Answer 2 (page = 127): If a batter runner is called out after missing home plate, the Official Scorer shall credit him with a three-base hit.
	Answer 3 (page = 126): (c) When the batter attempts to make a two-base hit or a three-base hit by sliding, he must hold the last base to which he advances.
	Answer 4 (page = 43): Rule 5.06(b)(4)(I) Comment: The fact

In [125]:
response.summary

summary_text: "In Major League Baseball, home base is a five-sided slab of whitened rubber that is 17 inches square [1, 2]. Two of the corners are removed so that one edge is 17 inches long, two adjacent sides are 8\302\275 inches, and the remaining two sides are 12 inches [1, 2]."

### Extractive Segments

Sections of text derived verbatim from documents in the search result.  Segments are longer than answers (previous section).  **These are good inputs for LLMs to use as context!!  Demonstrated later in this document**

[Extractive Segments](https://cloud.google.com/generative-ai-app-builder/docs/snippets#extractive-segments)

For now, Nov 2023, this feature using the [discoveryengine_v1alpha](https://cloud.google.com/python/docs/reference/discoveryengine/latest/google.cloud.discoveryengine_v1alpha.services.search_service.SearchServiceClient) client in order to get the `extractive_content_spec` attribute for the [discoveryengine_v1alpha1.types.SearchRequest.ContentSearchSpec()](https://cloud.google.com/python/docs/reference/discoveryengine/latest/google.cloud.discoveryengine_v1alpha.types.SearchRequest.ContentSearchSpec).

In [156]:
response = vertex_search_alpha.search(
    request = discoveryengine_alpha.SearchRequest(
        query = question,
        page_size = 10,
        serving_config = serving_config,
        content_search_spec = discoveryengine_alpha.SearchRequest.ContentSearchSpec(
            snippet_spec = discoveryengine_alpha.SearchRequest.ContentSearchSpec.SnippetSpec(
                return_snippet = True
            ),
            summary_spec = discoveryengine_alpha.SearchRequest.ContentSearchSpec.SummarySpec(
                summary_result_count = 5,
                include_citations = True,
                ignore_adversarial_query = True,
                ignore_non_summary_seeking_query = False
            ),
            extractive_content_spec = discoveryengine_alpha.SearchRequest.ContentSearchSpec.ExtractiveContentSpec(
                max_extractive_answer_count = 5,
                max_extractive_segment_count = 10,
                return_extractive_segment_score = True,
                num_previous_segments = 1,
                num_next_segments = 1
            )
        ),
        query_expansion_spec = discoveryengine_alpha.SearchRequest.QueryExpansionSpec(
            condition = discoveryengine_alpha.SearchRequest.QueryExpansionSpec.Condition.AUTO
        ),
        spell_correction_spec = discoveryengine_alpha.SearchRequest.SpellCorrectionSpec(
            mode = discoveryengine_alpha.SearchRequest.SpellCorrectionSpec.Mode.AUTO
        )
    )
)

In [157]:
text = ''
for r, result in enumerate(response.results):
    if r > 0: text += '\n\n\n'
    doc = dict(result.document.derived_struct_data)
    text += f"Source Document {r+1}:\n\tName: {doc['title']}\n\tLink: {doc['link']}"
    if 'snippets' in doc.keys():
        for s, snippet in enumerate(doc['snippets']):
            if snippet['snippet_status'] == 'SUCCESS':
                text += f"\n\tSnippet {s+1}: {snippet['snippet']}"
    #if 'extractive_answers' in doc.keys():
    #   for a, answer in enumerate(doc['extractive_answers']):
    #        text += f"\n\tAnswer {a+1} (page = {answer['pageNumber']}): {answer['content']}"
    if 'extractive_segments' in doc.keys():
        for e, extract in enumerate(doc['extractive_segments']):
            text += f"\n\tSegment {e+1} (page = {extract['pageNumber']}): {extract['content']}"
print(text)
segments_results = text

Source Document 1:
	Name: mlb_2022_rules
	Link: gs://statmike-mlops-349915/applied-genai/vertex-ai-search/mlb_2022_rules.pdf
	Snippet 1: ... <b>size</b> and shape of the grassed and bare areas of its playing field. NOTE: (a) ... The second <b>base</b> bag shall be centered on second <b>base</b>. The bags shall be 15&nbsp;...
	Segment 1 (page = 15): 3

Rule 2.01 to 2.03

The foul lines and all other playing lines indicated in the diagrams
by solid black lines shall be marked with paint or non-toxic and
non-burning chalk or other white material.
The grass lines and dimensions shown on the diagrams are those
used in many fields, but they are not mandatory and each Club shall
determine the size and shape of the grassed and bare areas of its
playing field.
NOTE: (a) Any Playing Field constructed by a professional
Club after June 1, 1958, shall provide a minimum distance of
325 feet from home base to the nearest fence, stand or other
obstruction on the right and left field foul lines, and a 

In [158]:
response.summary

summary_text: "In Major League Baseball, home base is a five-sided slab of whitened rubber that is 17 inches square [1, 2]. Two of the corners are removed so that one edge is 17 inches long, two adjacent sides are 8\302\275 inches, and the remaining two sides are 12 inches [1, 2]."

### Search Summaries

A `summary_spec` is provided with a extactive answer specification of only 1 answer - this creates an output in `response.summary` that is a paraphrased summary of the 

[Search Summaries](https://cloud.google.com/generative-ai-app-builder/docs/get-search-summaries)


Summaries are created with extractive answers, see above.  For now, Nov 2023, the extractive answers feature uses the [discoveryengine_v1alpha](https://cloud.google.com/python/docs/reference/discoveryengine/latest/google.cloud.discoveryengine_v1alpha.services.search_service.SearchServiceClient) client in order to get the `extractive_content_spec` attribute for the [discoveryengine_v1alpha1.types.SearchRequest.ContentSearchSpec()](https://cloud.google.com/python/docs/reference/discoveryengine/latest/google.cloud.discoveryengine_v1alpha.types.SearchRequest.ContentSearchSpec).

In [159]:
response = vertex_search_alpha.search(
    request = discoveryengine_alpha.SearchRequest(
        query = question,
        page_size = 10,
        serving_config = serving_config,
        content_search_spec = discoveryengine_alpha.SearchRequest.ContentSearchSpec(
            snippet_spec = discoveryengine_alpha.SearchRequest.ContentSearchSpec.SnippetSpec(
                return_snippet = True
            ),
            summary_spec = discoveryengine_alpha.SearchRequest.ContentSearchSpec.SummarySpec(
                summary_result_count = 5,
                include_citations = True,
                ignore_adversarial_query = True,
                ignore_non_summary_seeking_query = False
            ),
            extractive_content_spec = discoveryengine_alpha.SearchRequest.ContentSearchSpec.ExtractiveContentSpec(
                max_extractive_answer_count = 1
            )
        ),
        query_expansion_spec = discoveryengine_alpha.SearchRequest.QueryExpansionSpec(
            condition = discoveryengine_alpha.SearchRequest.QueryExpansionSpec.Condition.AUTO
        ),
        spell_correction_spec = discoveryengine_alpha.SearchRequest.SpellCorrectionSpec(
            mode = discoveryengine_alpha.SearchRequest.SpellCorrectionSpec.Mode.AUTO
        )
    )
)

In [160]:
response.summary

summary_text: "A base in baseball is a five-sided slab of whitened rubber that is 17 inches square [1, 2]. Two of the corners are removed so that one edge is 17 inches long, two adjacent sides are 8\302\275 inches, and the remaining two sides are 12 inches [1, 2]."
safety_attributes {
  categories: "Legal"
  scores: 0.10000000149011612
}

---
## Vertex LLM Setup

- TextGenerationModel [Guide](https://cloud.google.com/vertex-ai/docs/generative-ai/text/test-text-prompts)
    - TextGenerationModel [API](https://cloud.google.com/python/docs/reference/aiplatform/latest/vertexai.language_models.TextGenerationModel)

In [161]:
# create links to model: 
textgen_model = vertexai.language_models.TextGenerationModel.from_pretrained('text-bison@001')

Test text generation (llm) model:

In [162]:
question

'What is the size of a base?'

### Ask The LLM Without Context

The LLM likely has information about the question here from its vast pre-training data:

In [163]:
prediction = textgen_model.predict(question)
prediction

The size of a base is the area of the base. The area of a base is the amount of space that the base takes up. The area of a base is calculated by multiplying the length of the base by the width of the base.

In [164]:
prediction.text

'The size of a base is the area of the base. The area of a base is the amount of space that the base takes up. The area of a base is calculated by multiplying the length of the base by the width of the base.'

In [165]:
prediction.safety_attributes

{}

Ask for longer (potential) output:

In [166]:
textgen_model.predict(question, max_output_tokens = 500)

The size of a base is the area of the base. The area of a base is the amount of space that the base takes up. The area of a base is calculated by multiplying the length of the base by the width of the base.

### Ask The LLM With Context

Construct a prompt in three parts: setup, question, context

In [176]:
setup = 'I need help answering a question.  It needs to be answer from the context I provide.  The context is below for source documents.  Each source document is provided with key information: name, link, snippet(s), and segment(s).  The name is the file name.  The link is the URI of the document in Google Cloud storage.  Snippet(s) are small extracts from the document. Segment(s) are long passages from the document.  It there appears to be more than one answer, give both a cite the context part used.' 

In [177]:
prompt = f"{setup}\n\nContext:\n{segments_results}\n\nQuestion: {question}"

In [178]:
prediction = textgen_model.predict(prompt)

In [179]:
prediction.text

'The size of a base is 15 inches square, not less than three nor more than five inches thick, and filled with soft material.'

In [184]:
prediction.safety_attributes

{'Firearms & Weapons': 0.1, 'Legal': 0.2, 'Toxic': 0.1, 'War & Conflict': 0.1}

---
## Create A Function To Answer Questions

Use the extractive segments approach above.

In [257]:
def document_bot(question, show_context = False):
    # retrieve search with extactive segments:
    response = vertex_search_alpha.search(
        request = discoveryengine_alpha.SearchRequest(
            query = question,
            page_size = 10,
            serving_config = serving_config,
            content_search_spec = discoveryengine_alpha.SearchRequest.ContentSearchSpec(
                extractive_content_spec = discoveryengine_alpha.SearchRequest.ContentSearchSpec.ExtractiveContentSpec(
                    max_extractive_answer_count = 5,
                    max_extractive_segment_count = 10,
                    return_extractive_segment_score = True,
                    num_previous_segments = 2,
                    num_next_segments = 2
                )
            ),
            query_expansion_spec = discoveryengine_alpha.SearchRequest.QueryExpansionSpec(
                condition = discoveryengine_alpha.SearchRequest.QueryExpansionSpec.Condition.AUTO
            ),
            spell_correction_spec = discoveryengine_alpha.SearchRequest.SpellCorrectionSpec(
                mode = discoveryengine_alpha.SearchRequest.SpellCorrectionSpec.Mode.AUTO
            )
        )
    )
    
    # parse response segments into text string:
    context = []
    for r, result in enumerate(response.results):
        doc = dict(result.document.derived_struct_data)
        if 'extractive_segments' in doc.keys():
            for e, extract in enumerate(doc['extractive_segments']):
                context.append(extract['content'])
    context = ''.join(context)
    
    # construct prompt:
    prompt = f"Give a detailed answer to the question below using only this context: {context}.\nQuestion: {question}\nAnswer and Explanation:"
    
    # use LLM to answer question using prompt:
    prediction = textgen_model.predict(prompt)
    
    if show_context == True:
        answer = f'''## Response\n\n### Question\n{question}\n### Answer\n{prediction.text}\n### Sources\n'''
        
        text = ''
        for r, result in enumerate(response.results):
            if r > 0: text += '\n'
            doc = dict(result.document.derived_struct_data)
            text += f"{r+1}. {doc['title']} ({doc['link']}):"
            if 'extractive_segments' in doc.keys():
                for e, extract in enumerate(doc['extractive_segments']):
                    part = extract["content"][0:min([250,len(extract["content"])])].replace("\n", "")
                    text += f'''\n\t- Segment {e+1} (page = {extract["pageNumber"]}): {part}...'''
        
        IPython.display.display(IPython.display.Markdown(answer+text))
    else:
        return prediction.text

In [258]:
document_bot(question, show_context = False)

'The size of a base is 15 inches square, not less than three nor more than five inches thick, and filled with soft material.'

In [259]:
document_bot(question, show_context = True)

## Response

### Question
What is the size of a base?
### Answer
The size of a base is 15 inches square, not less than three nor more than five inches thick, and filled with soft material.
### Sources
1. mlb_2022_rules (gs://statmike-mlops-349915/applied-genai/vertex-ai-search/mlb_2022_rules.pdf):
	- Segment 1 (page = 15): 3Rule 2.01 to 2.03The foul lines and all other playing lines indicated in the diagramsby solid black lines shall be marked with paint or non-toxic andnon-burning chalk or other white material.The grass lines and dimensions shown on the diagram...
	- Segment 2 (page = 127): Rule 9.06(c) Comment: If the batter-runner overruns second orthird base and is tagged out trying to return, the Official Scorershall credit the batter-runner with the last base he touched. Ifa batter-runner runs past second base after reaching tha...
	- Segment 3 (page = 43): 31Rule 5.06(b)(4) to 5.06(c)(2)field, and is subsequently kicked or deflected into the dugout,stands or other area where the ball is dead, the awarding ofbases shall be two bases from position of runners at the timeof the pitch or throw.(I)  ...
	- Segment 4 (page = 126): 114Rule 9.06(b) to 9.06(c)an attempt to put out a preceding runner, the scorer shalldetermine whether the batter made a legitimate two-base hit orthree-base hit, or whether the batter-runner advanced beyondfirst base on the fielder’s choice.R...
	- Segment 5 (page = 127): 115Rule 9.06(c) to 9.06(f)hit; if the batter-runner overslides third base and is tagged out, theOfficial Scorer shall credit him with a two-base hit....
	- Segment 6 (page = 125): 113Rule 9.05(b)(1) to 9.06(b)(1)  runner is forced out by a batted ball, or would have beenforced out except for a fielding error;(2)  batter apparently hits safely and a runner who is forced toadvance by reason of the batter becoming a runner...
2. mlb_2023_rules (gs://statmike-mlops-349915/applied-genai/vertex-ai-search/mlb_2023_rules.pdf):
	- Segment 1 (page = 14): 2Rule 2.012.00–THE PLAYING FIELD2.01 Layout of the FieldThe field shall be laid out according to the instructions below,supplemented by the diagrams in Appendices 1, 2, and 3.The infield shall be a 90-foot square. The outfield shall be the a...
	- Segment 2 (page = 20): wide across the palm, measured from the base of the thumb crotch tothe outer edge of the mitt. The space between the thumb section andthe finger section of the mitt shall not exceed four inches at the top ofthe mitt and three and one-half inches a...
	- Segment 3 (page = 20): Thecrotch opening shall measure not more than 4½ inches at the top, not...
	- Segment 4 (page = 130): Rule 9.06(c) Comment: If the batter-runner overruns second orthird base and is tagged out trying to return, the Official Scorershall credit the batter-runner with the last base he touched. Ifa batter-runner runs past second base after reaching tha...
	- Segment 5 (page = 15): 3Rule 2.01 to 2.02The foul lines and all other playing lines indicated in the diagramsby solid black lines shall be marked with paint or non-toxic andnon-burning chalk or other white material.The grass lines and dimensions shown on the diagram...
	- Segment 6 (page = 129): 117Rule 9.06(b) to 9.06(c)an attempt to put out a preceding runner, the scorer shalldetermine whether the batter made a legitimate two-base hit orthree-base hit, or whether the batter-runner advanced beyondfirst base on the fielder’s choice.R...
	- Segment 7 (page = 130): 118Rule 9.06(c) to 9.06(f)hit; if the batter-runner overslides third base and is tagged out, theOfficial Scorer shall credit him with a two-base hit....
	- Segment 8 (page = 20): 8Rule 3.05 to 3.06...

In [260]:
document_bot("Is a rule broken if three infielders are positioned on the same side of the field where the batter is more likely to hit the ball?", show_context = True)

## Response

### Question
Is a rule broken if three infielders are positioned on the same side of the field where the batter is more likely to hit the ball?
### Answer
Yes, a rule is broken if three infielders are positioned on the same side of the field where the batter is more likely to hit the ball.

Rule 5.02(c) states that "at the time the pitcher releases the ball for delivery to
the batter, the defensive team must have a minimum
of four players (in addition to the pitcher and the
catcher) with both feet completely in front of the outer
boundary of the infield dirt, at least two of which must
be positioned with both feet entirely on each side of
second base; and
(iii) from the time
### Sources
1. mlb_2023_rules (gs://statmike-mlops-349915/applied-genai/vertex-ai-search/mlb_2023_rules.pdf):
	- Segment 1 (page = 31): 19Rule 5.02(c)(ii) at the time the pitcher releases the ball for delivery tothe batter, the defensive team must have a minimumof four players (in addition to the pitcher and thecatcher) with both feet completely in front of the outerboundary o...
	- Segment 2 (page = 128): 116Rule 9.05(b)(1) to 9.06(b)(1)  runner is forced out by a batted ball, or would have beenforced out except for a fielding error;(2)  batter apparently hits safely and a runner who is forced toadvance by reason of the batter becoming a runner...
	- Segment 3 (page = 36): 24Rule 5.04(c) to 5.05(a)(6)(c) Completing Time at BatA batter has legally completed his time at bat when he is put outor becomes a runner.5.05 When the Batter Becomes a Runner(a) The batter becomes a runner when:(1)  He hits a fair ball;R...
	- Segment 4 (page = 55): 43Rule 5.09(a)(6) to 5.09(a)(8)(6)  He attempts to hit a third strike and the ball touches him;(7)  His fair ball touches him before touching a fielder. If thebatter is in a legal position in the batter’s box, see Rule5.04(b)(5), and, in the u...
	- Segment 5 (page = 95): 83Rule 6.02(d) to 6.03(a)(4)Rule 6.02(d) Comment: If at any time the ball hits the rosin bag itis in play. In the case of rain or wet field, the umpire may instructthe pitcher to carry the rosin bag in his hip pocket. A pitcher mayuse the rosin...
2. mlb_2022_rules (gs://statmike-mlops-349915/applied-genai/vertex-ai-search/mlb_2022_rules.pdf):
	- Segment 1 (page = 42): 30Rule 5.06(b)(4)ball hit the ground, passes a receiving fielder or goes out of playinto the stands.The position of the batter-runner at the time the wild throw leftthe thrower’s hand is the key in deciding the award of bases.If the batter-run...
	- Segment 2 (page = 54): 42Rule 5.09(a)(8) to 5.09(a)(10)runners may advance. If the batter-runner drops his batand the ball rolls against the bat in fair territory and, inthe umpire’s judgment, there was no intention to interferewith the course of the ball, the ball i...
	- Segment 3 (page = 45): 33Rule 5.06(c)(7) to 5.07(a)Rule 5.06(c)(7) Comment: If a foul tip hits the umpire and iscaught by a fielder on the rebound, the ball is “dead” and thebatsman cannot be called out. The same shall apply where suchfoul tip lodges in the umpire’s ...
	- Segment 4 (page = 125): 113Rule 9.05(b)(1) to 9.06(b)(1)  runner is forced out by a batted ball, or would have beenforced out except for a fielding error;(2)  batter apparently hits safely and a runner who is forced toadvance by reason of the batter becoming a runner...
	- Segment 5 (page = 93): 81Rule 6.02(d) to 6.03(a)(4)Rule 6.02(d) Comment: If at any time the ball hits the rosin bag itis in play. In the case of rain or wet field, the umpire may instructthe pitcher to carry the rosin bag in his hip pocket. A pitcher mayuse the rosin...

In [261]:
document_bot("What is the definition of a balk?", show_context = True)

## Response

### Question
What is the definition of a balk?
### Answer
A balk is defined as:

(A) Straddling the pitcher’s rubber without the ball is to be
interpreted as intent to deceive and ruled a balk.
(B) With a runner on first base the pitcher may make a
complete turn, without hesitating toward first, and
throw to second. This is not to be interpreted as
throwing to an unoccupied base.
(C) The pitcher, while touching his plate, throws, or feints a
throw to an unoccupied base, except for the purpose of
making a play;
(D) The pitcher makes an illegal pitch;

### Sources
1. mlb_2022_rules (gs://statmike-mlops-349915/applied-genai/vertex-ai-search/mlb_2022_rules.pdf):
	- Segment 1 (page = 44): 32Rule 5.06(c)(2) to 5.06(c)(7)one or more bases as the result of acts which occurred while theball was alive (such as, but not limited to a balk, an overthrow,interference, or a home run or other fair ball hit out of the playingfield).Rule 5...
	- Segment 2 (page = 90): 78Rule 6.02(a) to 6.02(c)(3)(A) Straddling the pitcher’s rubber without the ball is to beinterpreted as intent to deceive and ruled a balk.(B) With a runner on first base the pitcher may make acomplete turn, without hesitating toward first, an...
	- Segment 3 (page = 139): 127Rule 9.12(e) to 9.13(a)first base when touched by a pitched ball, or when the batterreaches first base as the result of a wild pitch or passed ball.Rule 9.12(e) Comment: See Rule 9.13 for additional scoringrules relating to wild pitches and ...
	- Segment 4 (page = 88): 76Rule 6.02(a)(3) to 6.02(a)(8)Rule 6.02(a)(3) Comment: Requires the pitcher, while touchinghis plate, to step directly toward a base before throwing tothat base. If a pitcher turns or spins off of his free foot withoutactually stepping or if h...
	- Segment 5 (page = 46): 34Rule 5.07(a) to 5.07(a)(2)The pitcher may not take a second step toward home plate witheither foot or otherwise reset his pivot foot in his delivery of thepitch. If there is a runner, or runners, on base it is a balk underRule 6.02(a); if the...
2. mlb_2023_rules (gs://statmike-mlops-349915/applied-genai/vertex-ai-search/mlb_2023_rules.pdf):
	- Segment 1 (page = 92): 80Rule 6.02(a) to 6.02(c)(3)(A) Straddling the pitcher’s rubber without the ball is to beinterpreted as intent to deceive and ruled a balk.(B) With a runner on first base the pitcher may make acomplete turn, without hesitating toward first, an...
	- Segment 2 (page = 45): 33Rule 5.06(c)(1) to 5.06(c)(6)(1)  A pitched ball touches a batter, or his clothing, while in hislegal batting position; runners, if forced, advance;(2)  The plate umpire interferes with the catcher’s throwattempting to prevent a stolen base ...
	- Segment 3 (page = 142): 130Rule 9.12(e) to 9.13(a)first base when touched by a pitched ball, or when the batterreaches first base as the result of a wild pitch or passed ball.Rule 9.12(e) Comment: See Rule 9.13 for additional scoringrules relating to wild pitches and ...
	- Segment 4 (page = 90): 78Rule 6.02(a)(3) to 6.02(a)(8)Rule 6.02(a)(3) Comment: Requires the pitcher, while touchinghis plate, to step directly toward a base before throwing tothat base. If a pitcher turns or spins off of his free foot withoutactually stepping or if h...
	- Segment 5 (page = 48): 36Rule 5.07(a)(1) to 5.07(a)(2)(C) disengage the rubber (if he does he must drop his handto his sides).In disengaging the rubber the pitcher must step off with hispivot foot and not his free foot first. He may not go into a set orstretch posi...

In [262]:
document_bot("A batter hits a ball that goes over the fence, but it is caught by a fan in the stands. Is the ball a home run?", show_context = True)

## Response

### Question
A batter hits a ball that goes over the fence, but it is caught by a fan in the stands. Is the ball a home run?
### Answer
No, the ball is not a home run.

The rule states that a fair ball that passes over a fence or into the stands at a distance from home base of 250 feet or more. Such hit entitles the batter to a home run when he shall have touched all bases legally. A fair fly ball that passes out of the playing field at a point less than 250 feet from home base shall entitle the batter to advance to second base only.
### Sources
1. mlb_2023_rules (gs://statmike-mlops-349915/applied-genai/vertex-ai-search/mlb_2023_rules.pdf):
	- Segment 1 (page = 84): 72Rule 6.01(d) to 6.01(e)umpire should call interference on the part of the coach is upto the judgment of the umpire and if the umpire felt that thecoach did all he could to avoid interfering with the play, nointerference need be called. If, in...
	- Segment 2 (page = 43): 31Rule 5.06(b)(4)first. Batter hits fly to short right. Runner holds up between firstand second and batter comes around first and pulls up behindhim. Ball falls safely. Outfielder, in throwing to first, throwsball into stands.APPROVED RULING: ...
	- Segment 3 (page = 58): 46Rule 5.09(b)(2) to 5.09(b)(5)This rule also covers the following and similar plays: Less thantwo out, score tied last of ninth inning, runner on first, batterhits a ball out of park for winning run, the runner on first passessecond and thinki...
	- Segment 4 (page = 36): 24Rule 5.04(c) to 5.05(a)(6)(c) Completing Time at BatA batter has legally completed his time at bat when he is put outor becomes a runner.5.05 When the Batter Becomes a Runner(a) The batter becomes a runner when:(1)  He hits a fair ball;R...
	- Segment 5 (page = 52): 40Rule 5.08(b)Rule 5.08 Comment:APPROVED RULING: No run shall score during a playin which the third out is made by the batter-runner beforehe touches first base. Example: One out, Jones on second,Smith on first. The batter, Brown, hits safely...
2. mlb_2022_rules (gs://statmike-mlops-349915/applied-genai/vertex-ai-search/mlb_2022_rules.pdf):
	- Segment 1 (page = 35): 23Rule 5.05(a)(2) to 5.05(a)(8)(2)  The third strike called by the umpire is not caught,providing (1) first base is unoccupied, or (2) first base isoccupied with two out;Rule 5.05(a)(2) Comment: A batter who does not realize hissituation on a...
	- Segment 2 (page = 53): 41Rule 5.09(a)(1) to 5.09(a)(8)before it touches the ground. Runners may leave their basesthe instant the first fielder touches the ball. A fielder may reachover a fence, railing, rope or other line of demarcation to makea catch. He may jump on...
	- Segment 3 (page = 82): 70Rule 6.01(d) to 6.01(e)umpire should call interference on the part of the coach is upto the judgment of the umpire and if the umpire felt that thecoach did all he could to avoid interfering with the play, nointerference need be called. If, in...
	- Segment 4 (page = 36): 24Rule 5.05(a)(8) to 5.05(b)(2)which case the batter and all runners shall be entitled toadvance two bases;(9)  Any fair fly ball is deflected by the fielder into the stands,or over the fence into foul territory, in which case the battershall...
	- Segment 5 (page = 51): 39Rule 5.08(b)batter-runner before he touched first base, Jones’ run doesnot count.APPROVED RULING: Following runners are not affectedby an act of a preceding runner unless two are out.EXAMPLE: One out, Jones on second, Smith on first, and...

In [263]:
document_bot("What is the dimension of base?", show_context = True)

## Response

### Question
What is the dimension of base?
### Answer
The dimension of the base is 15 inches square, not less than three nor more than five inches thick, and filled with soft material.
### Sources
1. mlb_2022_rules (gs://statmike-mlops-349915/applied-genai/vertex-ai-search/mlb_2022_rules.pdf):
	- Segment 1 (page = 15): 3Rule 2.01 to 2.03The foul lines and all other playing lines indicated in the diagramsby solid black lines shall be marked with paint or non-toxic andnon-burning chalk or other white material.The grass lines and dimensions shown on the diagram...
	- Segment 2 (page = 127): Rule 9.06(c) Comment: If the batter-runner overruns second orthird base and is tagged out trying to return, the Official Scorershall credit the batter-runner with the last base he touched. Ifa batter-runner runs past second base after reaching tha...
	- Segment 3 (page = 126): 114Rule 9.06(b) to 9.06(c)an attempt to put out a preceding runner, the scorer shalldetermine whether the batter made a legitimate two-base hit orthree-base hit, or whether the batter-runner advanced beyondfirst base on the fielder’s choice.R...
	- Segment 4 (page = 43): 31Rule 5.06(b)(4) to 5.06(c)(2)field, and is subsequently kicked or deflected into the dugout,stands or other area where the ball is dead, the awarding ofbases shall be two bases from position of runners at the timeof the pitch or throw.(I)  ...
	- Segment 5 (page = 127): 115Rule 9.06(c) to 9.06(f)hit; if the batter-runner overslides third base and is tagged out, theOfficial Scorer shall credit him with a two-base hit....
	- Segment 6 (page = 125): 113Rule 9.05(b)(1) to 9.06(b)(1)  runner is forced out by a batted ball, or would have beenforced out except for a fielding error;(2)  batter apparently hits safely and a runner who is forced toadvance by reason of the batter becoming a runner...
2. mlb_2023_rules (gs://statmike-mlops-349915/applied-genai/vertex-ai-search/mlb_2023_rules.pdf):
	- Segment 1 (page = 15): 3Rule 2.01 to 2.02The foul lines and all other playing lines indicated in the diagramsby solid black lines shall be marked with paint or non-toxic andnon-burning chalk or other white material.The grass lines and dimensions shown on the diagram...
	- Segment 2 (page = 130): Rule 9.06(c) Comment: If the batter-runner overruns second orthird base and is tagged out trying to return, the Official Scorershall credit the batter-runner with the last base he touched. Ifa batter-runner runs past second base after reaching tha...
	- Segment 3 (page = 61): 49Rule 5.09(b)(9) to 5.09(b)(12)third base and home plate. Believing the lead runner willbe tagged out, the runner at second base (i.e., the trailingrunner) advances to third base. Before being tagged, the leadrunner runs back to and beyond th...
	- Segment 4 (page = 44): 32Rule 5.06(b)(4) to 5.06(c)APPROVED RULING: When a wild pitch or passed ball goesthrough or by the catcher, or deflects off the catcher, and goesdirectly into the dugout, stands, above the break, or any areawhere the ball is dead, the awarding...
	- Segment 5 (page = 129): 117Rule 9.06(b) to 9.06(c)an attempt to put out a preceding runner, the scorer shalldetermine whether the batter made a legitimate two-base hit orthree-base hit, or whether the batter-runner advanced beyondfirst base on the fielder’s choice.R...
	- Segment 6 (page = 130): 118Rule 9.06(c) to 9.06(f)hit; if the batter-runner overslides third base and is tagged out, theOfficial Scorer shall credit him with a two-base hit....